In [1]:
import pandas as pd
import numpy as np
import pickle
import Bio   
import plotly.express as px
import plotly.graph_objects as go
from jproperties import Properties
import wandb
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline, AutoModel, TrainingArguments, Trainer
from motif_utils import seq2kmer # Soruced from https://github.com/jerryji1993/DNABERT
import torch
from datasets import Dataset
import evaluate

c:\Users\zeusg\Envs\real-fast\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
'''
https://wandb.ai/pcoady/DNA-Weight/runs/vsv999jb?workspace=user-pcoady
'''

'\nhttps://wandb.ai/pcoady/DNA-Weight/runs/vsv999jb?workspace=user-pcoady\n'

In [3]:
#zhihan1996/DNA_bert_6

In [4]:
configs = Properties()
with open("env.properties", "rb") as config_file:
    configs.load(config_file)


In [5]:
def load_data():
    '''
    TODO:
     1. Split Gene's into 512 k-mer chunks
     2. Keep duplicate genes on either the eval or train side. 
    
    '''
    path = f"D:\DNAWeight\data\capstone_body_weight_and_sequences\capstone_dictionary_gene_fasta_sample_16122022.pkl"
    with open(configs.get("SEQ_LOCATION").data, 'rb') as file:
        dna_dict =  pickle.load(file)
    dna = pd.Series({k:seq2kmer(str(v.seq), 6) for k,v in dna_dict.items()})
    dna.name = "dna_seq"
    df = pd.read_csv(configs.get("BODY_WEIGHT_LOCATION").data, delimiter=";")
    df = df.merge(dna, left_on="marker_symbol", right_index=True)

    return df
df = load_data()

In [6]:
len(df.marker_symbol.unique())

424

In [7]:
df.head(10)

,parameter_name,allele_accession_id,zygosity,allele_symbol,marker_accession_id,marker_symbol,estimate_gene_effect_size_in_female,std_error_estimate_gene_effect_size_in_female,p_value_genotype_effect_in_female,adjusted_p_value_genotype_effect_in_female,estimate_gene_effect_size_in_male,std_error_estimate_gene_effect_size_in_male,p_value_genotype_effect_in_male,adjusted_p_value_genotype_effect_in_male,type_of_estimate,dna_seq
0,body weight,mgi:5443928,heterozygote,Trip13<tm1.1(KOMP)Vlcg>,MGI:1916966,Trip13,0.178826,0.100241,0.074477,0.313866,0.178826,0.100241,0.074477,0.192986,Overall genotype estimate,AAGAGA AGAGAA GAGAAT AGAATT GAATTC AATTCT ATTC...
1,body weight,mgi:5443928,homozygote,Trip13<tm1.1(KOMP)Vlcg>,MGI:1916966,Trip13,1.142868,0.239162,0.000002,0.000053,0.362294,0.236102,0.124962,0.388039,Sex-specific genotype estimate,AAGAGA AGAGAA GAGAAT AGAATT GAATTC AATTCT ATTC...
2,body weight,mgi:5446143,homozygote,Setd6<tm1.1(KOMP)Mbp>,MGI:1913333,Setd6,-0.131337,0.126265,0.298304,0.444773,-0.131337,0.126265,0.298304,0.436052,Overall genotype estimate,CTCTTC TCTTCT CTTCTC TTCTCT TCTCTC CTCTCT TCTC...
3,body weight,mgi:5448653,homozygote,Rad21l<tm1b(KOMP)Wtsi>,MGI:3652039,Rad21l,0.019190,0.131798,0.884239,0.904335,0.019190,0.131798,0.884239,0.904335,Overall genotype estimate,AAAGTT AAGTTA AGTTAA GTTAAA TTAAAT TAAATA AAAT...
4,body weight,mgi:5468968,homozygote,Tpgs2<tm1.1(KOMP)Vlcg>,MGI:1913898,Tpgs2,0.127253,0.109773,0.246400,0.431201,0.127253,0.109773,0.246400,0.418134,Overall genotype estimate,ATGGTT TGGTTC GGTTCA GTTCAG TTCAGC TCAGCT CAGC...
5,body weight,mgi:5474780,homozygote,Rimklb<tm1.1(KOMP)Vlcg>,MGI:1918325,Rimklb,0.309000,0.261218,0.236887,0.368490,-1.162641,0.251216,0.000004,0.000026,Sex-specific genotype estimate,TCATTC CATTCC ATTCCC TTCCCA TCCCAT CCCATT CCAT...
6,body weight,mgi:5494466,heterozygote,Lrp1<tm1.1(KOMP)Wtsi>,MGI:96828,Lrp1,0.219402,0.253100,0.386052,0.434308,-0.882094,0.250814,0.000440,0.003957,Sex-specific genotype estimate,TTCTTT TCTTTT CTTTTC TTTTCT TTTCTT TTCTTT TCTT...
7,body weight,mgi:5509058,heterozygote,Svep1<tm1b(EUCOMM)Hmgu>,MGI:1928849,Svep1,0.323476,0.129932,0.012815,0.102520,-0.511971,0.129932,0.000082,0.001316,Sex-specific genotype estimate,TATTTA ATTTAG TTTAGT TTAGTA TAGTAG AGTAGC GTAG...
8,body weight,mgi:5509325,homozygote,Igsf11<tm1b(KOMP)Wtsi>,MGI:2388477,Igsf11,0.055385,0.147537,0.707376,0.853644,0.565063,0.150647,0.000178,0.012444,Sex-specific genotype estimate,TCCTGG CCTGGA CTGGAA TGGAAC GGAACT GAACTC AACT...
9,body weight,mgi:5520180,heterozygote,Rab34<tm1b(EUCOMM)Hmgu>,MGI:104606,Rab34,-0.371636,0.116002,0.001363,0.020450,-0.371636,0.116002,0.001363,0.012270,Overall genotype estimate,CTCTCC TCTCCC CTCCCC TCCCCC CCCCCT CCCCTT CCCT...


In [9]:
df = df[["dna_seq","estimate_gene_effect_size_in_male"]]
df = df.rename({"estimate_gene_effect_size_in_male":"label"}, axis=1)

In [10]:
tokenizer = AutoTokenizer.from_pretrained('zhihan1996/DNA_bert_6')
model = AutoModelForSequenceClassification.from_pretrained('zhihan1996/DNA_bert_6', 
                                                           num_labels=1, 
                                                           ignore_mismatched_sizes=True)

Some weights of the model checkpoint at zhihan1996/DNA_bert_6 were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at zhihan1996/DNA_bert_6 a

In [11]:
def tokenize_function(df):
    return tokenizer(df["dna_seq"], padding=True, truncation=True, max_length=512)#512


In [12]:
dataset = Dataset.from_pandas(df)
tokenized_datasets = dataset.map(tokenize_function, batched=True)

100%|██████████| 1/1 [00:49<00:00, 49.19s/ba]


In [13]:
train, val, test = torch.utils.data.random_split(tokenized_datasets, [310, 66, 67])

In [14]:
print(tokenizer(df.head(1)["dna_seq"].values[0], padding=True, truncation=True, max_length=10))
print(len(tokenizer.decode(tokenizer(df.head(1)["dna_seq"].values[0], padding=True, truncation=True, max_length=512)["input_ids"])))
print(len(df.head(1)["dna_seq"].values[0]))

{'input_ids': [2, 209, 821, 3270, 778, 3099, 94, 364, 1441, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
3581
177379


In [16]:
3581/df["dna_seq"].apply(len).mean()

0.006322157869656096

In [12]:
wandb.init(project=configs.get("WNB_PROJECT").data, entity=configs.get("WNB_USER").data)
wandb_config = wandb.config

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: pcoady. Use `wandb login --relogin` to force relogin


In [13]:
mse_metric = evaluate.load("mse")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    
    wandb.log({'True Values': labels })
    wandb.log({'Predicted Values': logits})

    mse = mse_metric.compute(predictions=logits, references=labels)
    return mse

In [14]:
training_args = TrainingArguments(output_dir='weight_model', 
                                  evaluation_strategy='epoch',
                                  per_device_train_batch_size = 5,  
                                  num_train_epochs=100)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=val,
    compute_metrics=compute_metrics
)

In [15]:
trainer.train()

c:\Users\zeusg\Envs\real-fast\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 310
  Num Epochs = 100
  Instantaneous batch size per device = 5
  Total train batch size (w. parallel, distributed & accumulation) = 5
  Gradient Accumulation steps = 1
  Total optimization steps = 6200
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
  1%|          | 62/6200 [00:23<34:06,  3.00it/s] ***** Running Evaluation *****
  Num examples = 66
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, dna_seq. If __index_level_0__, dna_seq are not expect

{'eval_loss': 0.263769268989563, 'eval_mse': 0.2637692339813635, 'eval_runtime': 1.439, 'eval_samples_per_second': 45.865, 'eval_steps_per_second': 6.254, 'epoch': 1.0}


  2%|▏         | 124/6200 [00:45<34:09,  2.96it/s] ***** Running Evaluation *****
  Num examples = 66
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, dna_seq. If __index_level_0__, dna_seq are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
                                                  
  2%|▏         | 124/6200 [00:47<34:09,  2.96it/s]

{'eval_loss': 0.33650195598602295, 'eval_mse': 0.3365019577415374, 'eval_runtime': 1.459, 'eval_samples_per_second': 45.237, 'eval_steps_per_second': 6.169, 'epoch': 2.0}


  3%|▎         | 186/6200 [01:08<34:49,  2.88it/s]  ***** Running Evaluation *****
  Num examples = 66
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, dna_seq. If __index_level_0__, dna_seq are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
                                                  
  3%|▎         | 186/6200 [01:10<34:49,  2.88it/s]

...

 99%|█████████▉| 6138/6200 [39:35<00:23,  2.69it/s]***** Running Evaluation *****
  Num examples = 66
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, dna_seq. If __index_level_0__, dna_seq are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
                                                   
 99%|█████████▉| 6138/6200 [39:36<00:23,  2.69it/s]

{'eval_loss': 0.28854629397392273, 'eval_mse': 0.2885463020232283, 'eval_runtime': 1.565, 'eval_samples_per_second': 42.173, 'eval_steps_per_second': 5.751, 'epoch': 99.0}


100%|██████████| 6200/6200 [39:59<00:00,  2.75it/s]***** Running Evaluation *****
  Num examples = 66
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, dna_seq. If __index_level_0__, dna_seq are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
                                                   
100%|██████████| 6200/6200 [40:01<00:00,  2.75it/s]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 6200/6200 [40:01<00:00,  2.58it/s]

{'eval_loss': 0.2890647053718567, 'eval_mse': 0.2890646549190019, 'eval_runtime': 1.5625, 'eval_samples_per_second': 42.24, 'eval_steps_per_second': 5.76, 'epoch': 100.0}
{'train_runtime': 2401.3973, 'train_samples_per_second': 12.909, 'train_steps_per_second': 2.582, 'train_loss': 0.04743621633898827, 'epoch': 100.0}


TrainOutput(global_step=6200, training_loss=0.04743621633898827, metrics={'train_runtime': 2401.3973, 'train_samples_per_second': 12.909, 'train_steps_per_second': 2.582, 'train_loss': 0.04743621633898827, 'epoch': 100.0})

In [16]:
results = trainer.predict(test)

***** Running Prediction *****
  Num examples = 67
  Batch size = 8
The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, dna_seq. If __index_level_0__, dna_seq are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
 89%|████████▉ | 8/9 [00:01<00:00,  5.62it/s]

In [17]:
np.mean(np.abs(results.predictions.flatten() - results.label_ids))

0.38559186